# Link to working Colab Notebook:

If this doesn't work for you, try it in colab.

https://colab.research.google.com/drive/1T7lFV7xyOlWHkG1KkKEBnuvrpVgjGF3a?usp=sharing

# Install Dependencies

In [1]:
%pip install datasets
%pip install sentence-transformers
%pip install qdrant-client
%pip install gradio
%pip install imageio-ffmpeg
%pip install ffmpeg-python
%pip install imageio[ffmpeg]
%pip install moviepy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is 

# Load Dataset

In [2]:
from datasets import load_dataset

In [3]:
video_dataset = load_dataset("JohnVitz/CV_Final_Project_Video_With_Chunked_Captions_Bert_Topic_1")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

train-00000-of-00005.parquet:   0%|          | 0.00/182M [00:00<?, ?B/s]

train-00001-of-00005.parquet:   0%|          | 0.00/472M [00:00<?, ?B/s]

train-00002-of-00005.parquet:   0%|          | 0.00/453M [00:00<?, ?B/s]

train-00003-of-00005.parquet:   0%|          | 0.00/631M [00:00<?, ?B/s]

train-00004-of-00005.parquet:   0%|          | 0.00/358M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/8 [00:00<?, ? examples/s]

In [4]:
video_dataset = video_dataset['train']

# Initialize Embedding Model

In [5]:
import torch
from sentence_transformers import SentenceTransformer

# Select device ('cpu' or 'cuda' if available)
device = "cpu"  # Set to "cuda" if you want to use GPU (change to "cuda" to force GPU usage)
# Check if CUDA is available and if the user wants to use it
if torch.cuda.is_available() and device == "cuda":
    device = "cuda"
    print("Using GPU (CUDA)")
else:
    print("Using CPU")

# Load pre-trained Sentence Transformer model
sentence_model = SentenceTransformer('all-mpnet-base-v2', device=device)

Using CPU


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Initialize Qdrant Store

In [6]:
# Qdrant Vector Database
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams

In [7]:
qdrant_client = QdrantClient(location=":memory:")

In [8]:
MY_SHORT_COLLECTION = "short_search_collection"
MY_LONG_COLLECTION = "long_search_collection"

embedding_size = sentence_model.get_sentence_embedding_dimension()

first_collection = qdrant_client.create_collection(
    collection_name=MY_SHORT_COLLECTION,
    vectors_config=VectorParams(size=embedding_size, # Size of Snowflake Embedding Dimensions
                                distance=Distance.COSINE), # Cosine similarity for vector search
)

second_collection = qdrant_client.create_collection(
    collection_name=MY_LONG_COLLECTION,
    vectors_config=VectorParams(size=embedding_size, # Size of Snowflake Embedding Dimensions
                                distance=Distance.COSINE), # Cosine similarity for vector search
)

In [9]:
qdrant_client.get_collection(collection_name=MY_SHORT_COLLECTION)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), 

In [10]:
qdrant_client.get_collection(collection_name=MY_LONG_COLLECTION)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=0, segments_count=1, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=None, sharding_method=None, replication_factor=None, write_consistency_factor=None, read_fan_out_factor=None, on_disk_payload=None, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=None, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), 

# Upload the Segmentation Subsets

In [11]:
def load_qdrant_store_subset_short():

    # assuming `dataset` has columns: video_id, segments, segment_embeddings
    records = []

    for example in video_dataset:
        vid       = example["video_id"]
        segments  = example["segments"]            # List[{"topic","start","end","text","Name"}]
        embeddings = example["segment_embeddings"] # List[List[float]]

        # one record per segment
        for idx, (seg, emb) in enumerate(zip(segments, embeddings)):
            rec_id = f"{vid}_{idx}"              # concat video_id + segment index
            payload = {
                "video_id": vid,
                "segment_index": idx,
                "topic": seg["topic"],
                "start": seg["start"],
                "end": seg["end"],
                "text": seg["text"],
                "Name": seg["Name"],
            }
            records.append({
                #"id": rec_id,
                "vector": emb,
                "payload": payload
            })

    # Insert points into the collection
    from qdrant_client.models import Batch

    # Create a Batch object

    ids      = list(range(len(records)))
    vectors  = [r["vector"]  for r in records]
    payloads = [r["payload"] for r in records]

    my_batch = Batch(ids=ids, vectors=vectors, payloads=payloads)

    qdrant_client.upsert(
        collection_name=MY_SHORT_COLLECTION,
        points=my_batch
)

In [12]:
def load_qdrant_store_subset_long():

    # assuming `dataset` has columns: video_id, segments, segment_embeddings
    records = []

    for example in video_dataset:
        vid       = example["video_id"]
        segments  = example["segments2"]            # List[{"topic","start","end","text","Name"}]
        embeddings = example["segment_embeddings2"] # List[List[float]]

        # one record per segment
        for idx, (seg, emb) in enumerate(zip(segments, embeddings)):
            rec_id = f"{vid}_{idx}"              # concat video_id + segment index
            payload = {
                "video_id": vid,
                "segment_index": idx,
                "topic": seg["topic"],
                "start": seg["start"],
                "end": seg["end"],
                "text": seg["text"],
                "Name": seg["Name"],
            }
            records.append({
                #"id": rec_id,
                "vector": emb,
                "payload": payload
            })

    # Insert points into the collection
    from qdrant_client.models import Batch

    # Create a Batch object

    ids      = list(range(len(records)))
    vectors  = [r["vector"]  for r in records]
    payloads = [r["payload"] for r in records]

    my_batch = Batch(ids=ids, vectors=vectors, payloads=payloads)

    qdrant_client.upsert(
        collection_name=MY_LONG_COLLECTION,
        points=my_batch
)

In [13]:
load_qdrant_store_subset_short()

In [14]:
load_qdrant_store_subset_long()

In [15]:
def get_query_embedding(text):
    return sentence_model.encode(text)  # Encode the question (this returns a normalized embedding)

# Gets closest similarity score chunks
def get_context_chunks(norm_query_embedding, num_chunks=1, collection='short'):

    if collection == 'long':

        context_chunks = qdrant_client.query_points(
            collection_name=MY_SHORT_COLLECTION,
            query=norm_query_embedding,
            limit=num_chunks
        ).points
    else:
        context_chunks = qdrant_client.query_points(
            collection_name=MY_LONG_COLLECTION,
            query=norm_query_embedding,
            limit=num_chunks
        ).points


    #print(context_chunks)

    return context_chunks

In [16]:
import os
import gradio as gr
import tempfile
from datetime import datetime
from moviepy.editor import VideoFileClip # this is just "from moviepy import VideoFileClip" in original files

def to_seconds(ts: str) -> float:
    """Convert 'HH:MM:SS.sss' to seconds."""
    dt = datetime.strptime(ts, "%H:%M:%S.%f")
    return dt.hour*3600 + dt.minute*60 + dt.second + dt.microsecond/1e6

def get_payload_based_on_question(question, collection='short'):
    """
    Returns:
      text (str),                — the retrieved caption text
      video_id (str),            — video_id to reference the video in video_dataset
      start_ts (str),            — segment start timestamp
      end_ts (str)               — segment end timestamp
    """
    # 1) Embed & get top hit
    question_embedding = get_query_embedding(question)
    hit = get_context_chunks(question_embedding, collection='short')[0]
    payload = hit.payload
    #print("DEBUG payload:", payload)  # you can remove this after verifying

    # 2) Extract fields from payload
    text     = payload['text']
    video_id = payload['video_id']  # Extract video_id from payload
    start_ts = payload['start']
    end_ts   = payload['end']

    return text, video_id, start_ts, end_ts

def serve_video_and_text(question, collection):#, video_dataset):

    # 1) Pull the best‐matching segment payload
    text, video_id, start_ts, end_ts = get_payload_based_on_question(question, collection=collection)

    #print(f'\n\nVideo ID: {video_id}\n')
    #print(type(video_id))

    # 2) Look up the video data from the video_dataset using video_id
    video_row = video_dataset.filter(lambda x: x['video_id'] == video_id)  # Filter the dataset by video_id
    if len(video_row) == 0:
        raise ValueError(f"Video with video_id {video_id} not found in the dataset.")

    video_data = video_row['mp4'][0]  # Extract the binary video data (mp4)

    # 3) Write the video data to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp4") as temp_file:
        temp_file.write(video_data)  # Write the binary video data to the temporary file
        temp_video_path = temp_file.name  # Path to the temporary video file

    # 4) Extract the subclip from the video file
    segment = VideoFileClip(temp_video_path).subclip(
        to_seconds(start_ts),
        to_seconds(end_ts)
    )

    # 5) Save the subclip to a temporary file for further processing
    tmp = tempfile.NamedTemporaryFile(delete=False, suffix=".mp4")
    segment.write_videofile(tmp.name, audio_codec="aac", logger=None)
    segment.close()

    return tmp.name, text


  if event.key is 'enter':



In [ ]:
def create_video_interface():
    with gr.Blocks() as demo:
        gr.Markdown("### Video and Text Display Based on Question")

        question_input   = gr.Textbox(label="Ask a Question",
                                      placeholder="Enter a question related to the videos…")
        collection_input = gr.Radio(
            choices=['short', 'long'],
            value='short',
            label="Would you perfer your retrieved videos to be short, or long?"
        )

        video_output = gr.Video()
        text_output  = gr.Markdown()

        show_btn = gr.Button("Show")
        show_btn.click(
            fn=serve_video_and_text,
            inputs=[question_input, collection_input],
            outputs=[video_output, text_output]
        )

    demo.launch(share=True, debug=True)

create_video_interface()

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://04ccbbdeb3d5a4a9bf.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Filter:   0%|          | 0/8 [00:00<?, ? examples/s]